<a href="https://colab.research.google.com/github/Black-icee/Black-icee/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, session, send_from_directory
import joblib
import openai
import requests
import os

app = Flask(__name__)
app.secret_key = "your_secret_key"

# Load model and vectorizer
try:
    model = joblib.load('sqli_model_rf.pkl')
    vectorizer = joblib.load('vectorizer.pkl')
except FileNotFoundError:
    print("Error loading model or vectorizer.")
    model = None
    vectorizer = None

# Load payloads
def load_payloads(file_path='payloads.txt'):
    try:
        with open(file_path, 'r') as file:
            return [line.strip() for line in file if line.strip()]
    except FileNotFoundError:
        print("Payload file not found.")
        return []

# Infer SQLi type from payload
def infer_sqli_type(payload):
    if "UNION" in payload.upper():
        return "Union-Based"
    elif "OR" in payload.upper() or "AND" in payload.upper():
        return "Boolean-Based"
    elif "--" in payload or ";" in payload:
        return "Error-Based"
    else:
        return "Unknown"

# Assign severity based on SQLi type
def assign_severity_from_type(sqli_type):
    severity_map = {
        "Union-Based": "high",
        "Error-Based": "medium",
        "Boolean-Based": "low",
        "Unknown": "low"
    }
    return severity_map.get(sqli_type, "low")

# Check vulnerability
def check_vulnerability(url, payloads):
    headers = {"User-Agent": "SQLi Scanner"}
    results = []
    for payload in payloads:
        test_url = f"{url}?payload={requests.utils.quote(payload)}"
        try:
            response = requests.get(test_url, headers=headers, timeout=5)
            if model and vectorizer:
                transformed = vectorizer.transform([payload])
                prediction = model.predict(transformed)[0]
                if prediction == 1:
                    sqli_type = infer_sqli_type(payload)
                    severity = assign_severity_from_type(sqli_type)
                    results.append({
                        "payload": payload,
                        "type": sqli_type,
                        "severity": severity
                    })
        except requests.RequestException:
            continue
    return results

# OpenAI API key
api_key = ""
if os.path.exists('../..openai_secret'):
    with open('../..openai_secret', 'r') as f:
        api_key = f.read().strip()

openai.api_key = api_key

# Generate AI report (payloads hidden from UI, used in prompt)
def generate_report(vulnerabilities):
    if not api_key:
        return "Missing OpenAI API key.", None

    prompt = (
        "A SQL injection scan detected vulnerabilities. Below are the findings:\n\n" +
        "\n".join([
            f"- Severity: {v['severity'].capitalize()}\n  Type: {v['type']}"
            for v in vulnerabilities
        ]) +
        "\n\nPlease:\n"
        "1. Group the findings by severity level (High, Medium, Low).\n"
        "2. For each, explain the potential impact.\n"
        "3. Suggest detailed and actionable remediation steps.\n"
        "4. Provide general best practices for SQLi prevention."
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000
        )
        content = response.choices[0].message['content'].strip()
        return content, content
    except Exception as e:
        return f"Error generating report: {e}", None

# Home route
@app.route("/", methods=["GET", "POST"])
def index():
    scan_limit = 2
    if "scans" not in session:
        session["scans"] = 0

    report = ""
    message = ""
    download_link = None

    if request.method == "POST":
        if session["scans"] >= scan_limit:
            message = "Scan limit reached. (2 scans per session)"
        else:
            url = request.form["url"]
            payloads = load_payloads()
            findings = check_vulnerability(url, payloads)
            session["scans"] += 1

            if findings:
                report_html, report_text = generate_report(findings)
                report = report_html

                # Save report
                filename = "sql_report.txt"
                filepath = os.path.join("static/reports", filename)
                os.makedirs(os.path.dirname(filepath), exist_ok=True)
                with open(filepath, "w") as f:
                    f.write(report_text)
                download_link = filepath
            else:
                message = "No vulnerabilities detected."

    return render_template(
        "index.html",
        report=report,
        message=message,
        download_link=download_link,
        remaining=scan_limit - session["scans"]
    )

# Report download route (optional, not used in HTML)
@app.route("/download/<filename>")
def download(filename):
    return send_from_directory("static/reports", filename, as_attachment=True)

if __name__ == "__main__":
    app.run(debug=True)
